## Grammar checker using Deep learning approach

In [1]:
from transformers import T5Tokenizer
from datasets import Dataset
import sentencepiece
import pandas as pd

In [7]:
from transformers import BertTokenizer
from datasets import Dataset
import pandas as pd

# Load the BERT tokenizer (you can use any BERT variant)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Sample data: incorrect and correct Tamil sentences (expand this with more examples)
data = {
    "incorrect_sentence": [
        "நான் பள்ளிக்குப் போகின்றன",
        "அவன் காபி குடிக்கின்றா"
    ],
    "correct_sentence": [
        "நான் பள்ளிக்குப் போகின்றேன்",
        "அவன் காபி குடிக்கின்றான்"
    ]
}

# Convert to pandas dataframe
df = pd.DataFrame(data)

# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Tokenization function
def tokenize_function(examples):
    inputs = tokenizer(examples['incorrect_sentence'], padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(examples['correct_sentence'], padding="max_length", truncation=True, max_length=128)
    inputs['labels'] = targets['input_ids']
    return inputs

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Check the tokenized dataset (Optional)
print(tokenized_dataset[0])


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Hp\\.cache\\huggingface\\hub\\models--bert-base-uncased\\snapshots\\86b5e0934494bd15c9632b12f734a8a67f723594\\tokenizer_config.json'

In [ ]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# Load pre-trained T5 model
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    save_steps=1000,
    evaluation_strategy="steps"
)

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  # You can use a separate validation set here
)

trainer.train()

# Save the model after training
model.save_pretrained("./grammar-correction-model")
tokenizer.save_pretrained("./grammar-correction-model")


In [ ]:
def correct_sentence(incorrect_sentence):
    inputs = tokenizer(incorrect_sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)
    outputs = model.generate(inputs['input_ids'], max_length=128, num_beams=5, early_stopping=True)
    corrected_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_sentence

# Example usage
incorrect_sentence = "நான் பள்ளிக்குப் போகின்றன"
corrected = correct_sentence(incorrect_sentence)
print(corrected)  # Output: "நான் பள்ளிக்குப் போகின்றேன்"
